In [1]:
import numpy as np

In [2]:
def get_alphabet(option="english"):
    if option == "english":
        return list(map(chr, range(ord("a"), ord("z")+1)))
    elif option == "russian":
        return list(map(chr, range(ord("а"), ord("я")+1)))

In [3]:
def marchroute_cipher(message: str, key: str):
    alphabet_russian = get_alphabet("russian")
    alphabet_english = get_alphabet()
    columns_size = len(key)
    
    message_cleared = list(filter(lambda s: s.lower() in alphabet_russian or s in alphabet_english, message))
    
    message_matrix = [
        [letter for letter in message_cleared[i:i+columns_size]] 
        for i in range(0, len(message_cleared), columns_size)
    ]
    
    if len(message_matrix[-1]) < columns_size:
        message_matrix[-1] = message_matrix[-1] +
            [message_matrix[-1][-1]]*(columns_size-len(message_matrix[-1]))
    
    message_password_dict = { value : np.array(message_matrix)[:,k] for k, value in enumerate(list(key)) }
    
    ciphered_message = ''.join([''.join(message_password_dict[k]).upper() 
                                for k in sorted(message_password_dict.keys())])
    
    return ciphered_message

In [4]:
m_test = "нельзя недооценивать противника"
k_test = "пароль"

In [5]:
result = marchroute_cipher(m_test, k_test)
print(f'Результат шифрования: \
        \n{m_test} * [{k_test}]\n-> {result, len(result)}')

Результат шифрования:         
нельзя недооценивать противника * [пароль]
-> ('ЕЕНПНЗОАТАЬОВОКННЕЬВЛДИРИЯЦТИА', 30)


In [6]:
# Не удалось реализовать
def cell_cipher(message: str, key: str, matrix_key=None):
    alphabet_russian = get_alphabet("russian")
    alphabet_english = get_alphabet()
    columns_size = len(key)
    
    message_cleared = list(filter(lambda s: s.lower() in alphabet_russian or s in alphabet_english, message))
    
    k = round(len(key) ** (1/2))

    message_matrix = np.matrix([
        [i+1 for i in range(ki, ki+k)] 
        for ki in range(0, k**2, k)
    ])
    
    mask_matrix = [
        np.asarray(np.rot90(message_matrix, k = i, axes = (1,0))).flatten() for i in range(0, 4)
    ]
    
    matrix_key = [np.zeros(k**2) for i in range(0, k**2)]
    
    for i in range(0, k**2):
        bit_mask = (mask_matrix[i%4] == i+1).astype(int)
        #print(bit_mask)
        matrix_key[i%4] = mask_matrix[i%4].multiply(bit_mask)
    
    return matrix_key

In [7]:
#m_test = "договор подписали"
#k_test = "сообщение"

In [8]:
#test_cell_cipher = cell_cipher(m_test, k_test)
#print(test_cell_cipher)

In [9]:
def vigenere_table(message: str, key: str, differ_alphabet=False):
    alphabet_russian = get_alphabet("russian")
    if differ_alphabet:
        alphabet_russian.remove('ь')
        alphabet_russian[alphabet_russian.index('ъ')] = 'ь'
    alphabet_english = get_alphabet()
    
    def find_letter_for_pair(letters_pair: tuple):
        if letters_pair[0].lower() in alphabet_russian:
            orig_letter_index = alphabet_russian.index(letters_pair[1].lower())
            key_letter_index = alphabet_russian.index(letters_pair[0].lower())
            
            shift = orig_letter_index + key_letter_index
            
            if shift > len(alphabet_russian):
                return alphabet_russian[shift - len(alphabet_russian)]
            
            return alphabet_russian[shift]
    
    message_cleared = list(filter(lambda s: s.lower() in alphabet_russian or s in alphabet_english, message))
    row_length = len(message_cleared)
    full_key = (list(key) * row_length)[:row_length]
    
    message_key_zip = list(zip(full_key, message_cleared))
    
    return ''.join(list(map(find_letter_for_pair, message_key_zip))).upper()

In [10]:
m_test = "криптография - серьезная наука"
k_test = "математика"

In [11]:
result = vigenere_table(m_test, k_test, True)

print(f'Результат шифрования: \
        \n({m_test}) * [{k_test}]\n-> {result, len(result)}')

Результат шифрования:         
(криптография - серьезная наука) * [математика]
-> ('ЦРЬФЯОХШКФФЯДКЭЬЧПЧАЛНТШЦА', 26)
